In [3]:
from alpaca.data import StockHistoricalDataClient, TimeFrame
from alpaca.data.requests import StockQuotesRequest, StockBarsRequest
import pandas as pd
import openpyxl, os

# Set your Alpaca API key and secret
ALPACA_KEY = 'AKUTKY5V16DQ533MCM95'
ALPACA_SECRET = 'lusSAgoBe63lD7BI3EZA74LrGMTUeR6dD5I67DeV'
BASE_URL = 'https://api.alpaca.markets'  # Use 'https://api.alpaca.markets' for live trading

start_time = pd.to_datetime("2023-01-19 09:30:00").tz_localize('America/New_York')
end_time = pd.to_datetime("2023-11-29 23:59:00").tz_localize('America/New_York')

# Instantiate a data client
data_client = StockHistoricalDataClient(ALPACA_KEY, ALPACA_SECRET)

request_params = StockBarsRequest(
    symbol_or_symbols=['SPY'],
    timeframe=TimeFrame.Day,
    start=start_time
    )

bars_df = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)
print(bars_df)




                                    open      high     low   close  \
symbol timestamp                                                     
SPY    2023-01-20 00:00:00-05:00  390.10  396.0400  388.38  395.88   
       2023-01-23 00:00:00-05:00  396.72  402.6450  395.72  400.63   
       2023-01-24 00:00:00-05:00  398.88  401.1500  397.64  400.20   
       2023-01-25 00:00:00-05:00  395.95  400.7000  393.56  400.35   
       2023-01-26 00:00:00-05:00  403.13  404.9200  400.03  404.75   
...                                  ...       ...     ...     ...   
       2023-12-04 00:00:00-05:00  455.60  459.1206  454.34  456.69   
       2023-12-05 00:00:00-05:00  455.26  457.5900  454.87  456.60   
       2023-12-06 00:00:00-05:00  458.81  458.8400  454.31  454.76   
       2023-12-07 00:00:00-05:00  456.91  458.9000  456.29  458.23   
       2023-12-08 00:00:00-05:00  457.46  460.7450  457.21  460.20   

                                      volume  trade_count        vwap  
symbol timestamp 

In [4]:
############################Using data from Alpaca ###############################################
from alpaca_trade_api.rest import REST, TimeFrame
import pandas_ta as ta
import pandas as pd
import openpyxl, os
from matplotlib import pyplot as plt
import matplotlib.colors 
from datetime import date
from datetime import datetime, timedelta

# Set your Alpaca API key and secret
ALPACA_KEY = 'AKUTKY5V16DQ533MCM95'
ALPACA_SECRET = 'lusSAgoBe63lD7BI3EZA74LrGMTUeR6dD5I67DeV'
BASE_URL = 'https://api.alpaca.markets'  # Use 'https://api.alpaca.markets' for live trading

arg1 = 'RIVN'
rest_api = REST(ALPACA_KEY, ALPACA_SECRET, base_url=BASE_URL)

# Create a folder named 'data' in the current directory
folder_path = './data'
os.makedirs(folder_path, exist_ok=True)
excel_file = os.path.join(folder_path, arg1 + '_ta_data_alpaca.xlsx')
base_excel_file = os.path.join(folder_path, arg1 + '_base_ta_data_alpaca.xlsx')

def retrieve_data(arg1):
    # retrieve daily bar data for SPY in a dataframe
    d = datetime.today() - timedelta(days=365)
    d2 = datetime.today() - timedelta(days=1)
    dstr = d.strftime("20%y-%m-%d")
    d2str = d2.strftime("20%y-%m-%d")
    df = rest_api.get_bars(arg1, TimeFrame.Day, dstr, d2str).df
    # Extract date from timestamp and set it as the first column
    df['date'] = df.index.date
    # Reorder columns with 'date' as the first column
    df = df[['date', 'close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap']]
    return df
    #return df.sort_values(by='date', ascending=False)
       
# METHODS GENERATING THE VARIOUS TECHNICAL PLOTS
def gen_indicators(arg1,excel_file):
    stock_bars = retrieve_data(arg1)
    stock_bars.to_excel(base_excel_file, index=False)
    ########### gen_moving_averages
    df_sma20 = stock_bars.ta.ema(length=20)
    df_sma50 = stock_bars.ta.sma(length=50)
    df_sma80 = stock_bars.ta.sma(length=80)
    df_hma20 = stock_bars.ta.hma(length=20)
    df_hma50 = stock_bars.ta.hma(length=50)
    ########### def gen_volatility
    df_acc50 = stock_bars.ta.accbands()
    df_acc50['date'] = df_acc50.index.date
    # Reorder columns with 'date' as the first column
    df_acc50 = df_acc50[['date','ACCBL_20','ACCBM_20','ACCBU_20']]
            
    rvi = stock_bars.ta.rvi() ## this a time series object and can be directly added to the stock_bars dataframe
    ## so there is no need to conver to dataframe
    ########## def gen_momentum   
    df_macd = stock_bars.ta.macd()
    df_macd['date'] = df_macd.index.date
    df_macd = df_macd[['date','MACD_12_26_9',  'MACDh_12_26_9', 'MACDs_12_26_9' ]]
    rsi = stock_bars.ta.rsi()
    ########## def gen_trend
    df_vortex = stock_bars.ta.vortex()
    df_vortex['date'] = df_vortex.index.date
    df_vortex = df_vortex [['date','VTXP_14', 'VTXM_14']]
    
    df_adx = stock_bars.ta.adx()
    df_adx['date'] = df_adx.index.date
    df_adx = df_adx [['date','ADX_14','DMP_14','DMN_14']]
    
    ##############################
    stock_bars['sma20'] = df_sma20
    stock_bars['sma50'] = df_sma50
    stock_bars['sma80'] = df_sma80
    stock_bars['hma20'] = df_hma20
    stock_bars['hma50'] = df_hma50
    stock_bars['rvi'] = rvi
    stock_bars['RSI_14'] = rsi
    ##############################
    #### now there are  5 data frames stock_bars,df_adx,df_vortex,df_macd,df_acc50
    #### we will write these to 5 different sheets within the excel workbook   
    ######### arrange the dataframe
    stock_bars = stock_bars[['date', 'close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap','sma20','sma50','sma80','hma20','hma50','rvi','RSI_14']]
    #print(stock_bars.info())
    #stock_bars.to_excel(excel_file, index=False)
    # Create an ExcelWriter object and specify the Excel file path
    
    with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
         # Write each DataFrame to a different sheet
         stock_bars.to_excel(writer, sheet_name='movAvgs_rsi_rvi', index=False)
         df_adx.to_excel(writer, sheet_name='adx', index=False)
         df_vortex.to_excel(writer, sheet_name='vortex', index=False)
         df_macd.to_excel(writer, sheet_name='macd', index=False)
         df_acc50.to_excel(writer, sheet_name='acc50', index=False)
          
gen_indicators(arg1,excel_file)


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from talib import RSI, BBANDS, MACD
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from talib import RSI, BBANDS, MACD

sns.set_style('whitegrid')
idx = pd.IndexSlice
